In [ ]:
import pandas as pd
import ast
import datetime
pd.set_option('display.max_row', 1000)
data = pd.read_csv("/Users/l.sergeev/Downloads/3495000.customEvents.c835e361950bd65959934461d11b9956.0.csv")
data.info()

In [ ]:
new_version_users = data.loc[data.idAppVersionTitle.str.contains("5.1")].reset_index(drop=True)
new_version_users

In [ ]:
def event_parser(df):
    names = df['params.name'][0]
    names = ast.literal_eval(names)
    df['params.value'] = df['params.value'].astype(str).apply(lambda x: ast.literal_eval(x))
    df1 = pd.DataFrame(df["params.value"].to_list(), columns=names)
    df2 = df.join(df1)
    return df2

In [ ]:
new_version_users_add_parsed = event_parser(new_version_users)

In [ ]:
new_version_users_add_parsed

In [ ]:
users_add_track = new_version_users_add_parsed.groupby(['customUserId','dtEvent','actions','type','subscription_type'])['countCustomEvent'].sum().reset_index()
users_add_track

In [ ]:
users_add_track_track_only = users_add_track[users_add_track['type'] == 'track']
users_add_track_track_only

In [ ]:
users_likes = users_add_track_track_only[(users_add_track_track_only['actions'] == 'like') & (users_add_track_track_only['subscription_type'] == 'active' )]
users_likes = users_likes.groupby(['customUserId','actions'])['countCustomEvent'].sum().reset_index()
users_likes.columns = ['customUserId', 'like','liked_items']
users_likes

In [ ]:
users_cache = users_add_track_track_only[(users_add_track_track_only['actions'] == 'cache') & (users_add_track_track_only['subscription_type'] == 'active' )]
users_cache = users_cache.groupby(['customUserId','actions'])['countCustomEvent'].sum().reset_index()
users_cache.columns = ['customUserId', 'cache','cached_items']
users_cache

In [ ]:
users_likes.drop_duplicates()

In [ ]:
_____________________________

In [ ]:
data_tracks = pd.read_csv("/Users/l.sergeev/Downloads/result_Play_track_to_end_2021-01-18.csv")


In [ ]:
data_tracks

In [ ]:
data_tracks_w_likes = data_tracks.merge(users_likes,on='customUserId',how='left')
data_tracks_w_likes

In [ ]:
data_tracks_w_likes['like'] = data_tracks_w_likes['like'].fillna('Not')
data_tracks_w_likes

In [ ]:
data_tracks_w_cache = data_tracks_w_likes.merge(users_cache,on='customUserId',how='left')
data_tracks_w_cache['cache'] = data_tracks_w_cache['cache'].fillna('Not')



In [ ]:
data_tracks_w_cache

In [ ]:
data_tracks_w_cache = data_tracks_w_cache[data_tracks_w_cache['subscription_type'] == 'active']
data_tracks_w_cache

In [ ]:
data_tracks_w_cache_grouped = data_tracks_w_cache.groupby(['customUserId','cache','progress','like'])['countCustomEvent'].sum().reset_index()
data_tracks_w_cache_grouped['time'] = data_tracks_w_cache_grouped['progress'].astype(int) / 100 * 205 * data_tracks_w_cache_grouped['countCustomEvent']
data_tracks_w_cache_grouped

In [ ]:
#data_tracks_w_cache_grouped_not = data_tracks_w_cache_grouped[data_tracks_w_cache_grouped['cache'] == 'cache']
data_tracks_w_cache_grouped_not = data_tracks_w_cache_grouped[(data_tracks_w_cache_grouped['like'] == 'like') & (data_tracks_w_cache_grouped['cache'] == 'cache' )]
tracks = data_tracks_w_cache_grouped_not.countCustomEvent.sum()
time = data_tracks_w_cache_grouped_not.time.sum()
users = data_tracks_w_cache_grouped_not.customUserId.nunique()
total = tracks / users
total2 = time / users
print(total)
print(total2)
print(users)


In [ ]:
test_wo_null = data_tracks_w_cache_grouped_not[~data_tracks_w_cache_grouped_not["customUserId"].str.contains('null')]
test_wo_null


In [ ]:
tracks = test_wo_null.countCustomEvent.sum()
time = test_wo_null.time.sum()
users = test_wo_null.customUserId.nunique()
total = tracks / users
total2 = time / users
print(total)
print(total2)
print(users)

In [ ]:
test_wo_null_grouped = test_wo_null.groupby(['customUserId'])['countCustomEvent'].sum().reset_index()
test_wo_null_grouped

In [ ]:
test_wo_null_grouped.to_csv("/Users/l.sergeev/Downloads/test_wo_null_grouped.csv")

In [ ]:
xe = data_tracks_mix.countCustomEvent.sum()

In [ ]:
ye = data_tracks_mix.customUserId.nunique()

In [ ]:
xe / ye

In [ ]:
___________________________

In [ ]:
data_tracks_subs = data_tracks[data_tracks['subscription_type'] == 'active']
data_tracks_subs

In [ ]:
cat = pd.read_excel("/Users/l.sergeev/Downloads/cat.xlsx",sheet_name = 'Sheet1')

In [ ]:
data_tracks_subs_cat = data_tracks_subs.merge(cat,on='from',how='right')
data_tracks_subs_cat

In [ ]:
mix_users_df = data_tracks_subs_cat[['customUserId']][data_tracks_subs_cat['normal_title'] == 'Микс']
mix_users_df = mix_users_df.drop_duplicates().dropna()
mix_users_df['Mix_user'] = 'Mix_user'
mix_users_df

In [ ]:
df4_main = data_tracks_subs_cat.merge(mix_users_df,on=['customUserId'],how='left')
df4_main = df4_main[df4_main['dtEvent'].notna()]
df4_main['Mix_user'] = df4_main['Mix_user'].fillna('Non mix user')
df4_main

In [ ]:
data_tracks_mix = df4_main.groupby(['customUserId','Mix_user','progress'])['countCustomEvent'].sum().reset_index()
data_tracks_mix['time'] = data_tracks_mix['progress'].astype(int) / 100 * 205 * data_tracks_mix['countCustomEvent']
data_tracks_mix

In [ ]:
data_tracks_mix_final = data_tracks_mix[data_tracks_mix['Mix_user'] == 'Non mix user']
tracks = data_tracks_mix_final.countCustomEvent.sum()
time = data_tracks_mix_final.time.sum()
users = data_tracks_mix_final.customUserId.nunique()
total = tracks / users
total2 = time / users
print(total)
print(total2)
print(users)